In [40]:
import numpy as np
import pandas as pd
import json

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Dataset

In [57]:
df =pd.read_csv("drive/My Drive/ColabDrive/Movie_Prediction/movies_metadata.csv")

#df = pd.read_csv("movies_metadata.csv")
df.head()

<ipython-input-57-f484e091a95d>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv("drive/My Drive/ColabDrive/Movie_Prediction/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### I chose 4 categories from the original dataset.
### Among English language movies, I will have a look at movieID, title and its genres

In [43]:
df = df[['id', 'original_title', 'original_language', 'genres']]
df = df.rename(columns={'id':'movieId'})
df = df[df['original_language'] == 'en']
df.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


### Due to huge dataset size, I used samll ratings dataset.

In [44]:
ratings = pd.read_csv("drive/My Drive/ColabDrive/Movie_Prediction/ratings_small.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [45]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


# Refine Dataset

In [46]:
# change string to numeric
df.movieId = pd.to_numeric(df.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

In [47]:
# genre displays as json string, and the function convert to list for ease use.
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))

    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

df['genres'] = df['genres'].apply(parse_genres)

df.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[Animation, Comedy, Family]"
1,8844,Jumanji,en,"[Adventure, Fantasy, Family]"
2,15602,Grumpier Old Men,en,"[Romance, Comedy]"
3,31357,Waiting to Exhale,en,"[Comedy, Drama, Romance]"
4,11862,Father of the Bride Part II,en,[Comedy]


# Merge DF and Ratings

In [48]:
# Merge User ratings table and movie table
data = pd.merge(ratings, df, on='movieId', how='inner')

data.head()

,userId,movieId,rating,original_title,original_language,genres
0,1,1371,2.5,Rocky III,en,[Drama]
1,4,1371,4.0,Rocky III,en,[Drama]
2,7,1371,3.0,Rocky III,en,[Drama]
3,19,1371,4.0,Rocky III,en,[Drama]
4,21,1371,3.0,Rocky III,en,[Drama]


# Pivot Table

In [49]:
# The pivot table converts the dataset into a user-item matrix format.
# In this matrix, rows represent individual users, columns represent movies (items), and the cells contain ratings that users have given to movies.
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head(10)

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pearson Correlation


*   Based on User Ratings, I used Pearson Correlation to measure the similarities between similar user rating movies.
*   If those movies share similar genre, then add more weights on correlation.



In [50]:
GENRE_WEIGHT = 0.1

# The larger the value of s1_c * s2_c, the larger similarities.
def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    ratings_sum = np.sum(s1_c * s2_c)

    return ratings_sum / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))



In [51]:
def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = df[df['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []
    for title in matrix.columns:
        if title == input_movie:
            continue

        # rating comparison
        # As the value of cor increases, it has more linear similarities.
        cor = pearsonR(matrix[input_movie], matrix[title])

        # genre comparison
        if similar_genre and len(input_genres) > 0:
            temp_genres = df[df['original_title'] == title]['genres'].iloc(0)[0]
            # compare the current movie genre and input movie genre
            # More identical genre in two movies, then add weight on cor
            same_count = np.sum(np.isin(input_genres, temp_genres))
            cor += (GENRE_WEIGHT * same_count)

        # skip NaN values in the cell
        if np.isnan(cor):
            continue
        else:
            result.append((title, '{:.2f}'.format(cor), temp_genres))

    # sort the list in decreasing order
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

# KNN Algorithms

In [52]:
def find_nearest_neighbors_pearson(input_movie, matrix, k):
    similarities = {}
    for title in matrix.columns:
        if title == input_movie:
            continue

        # Compute Pearson Correlation
        correlation = pearsonR(matrix[input_movie], matrix[title])
        if np.isnan(correlation):
            continue

        similarities[title] = correlation

    # Sort and return top k similar movies
    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:k]

def recommend_knn_pearson(input_movie, matrix, df, k, similar_genre=True):
    nearest_neighbors = find_nearest_neighbors_pearson(input_movie, matrix, k)
    result = []

    input_genres = df[df['original_title'] == input_movie]['genres'].iloc(0)[0]
    for title, similarity in nearest_neighbors:
        if similar_genre:
            temp_genres = df[df['original_title'] == title]['genres'].iloc(0)[0]
            same_count = np.sum(np.isin(input_genres, temp_genres))
            similarity += GENRE_WEIGHT * same_count

        result.append((title, similarity, temp_genres))

    result.sort(key=lambda x: x[1], reverse=True)
    return result

In [53]:
# prediction using Pearson Correlation only
recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)
PC_prediction = pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])
PC_prediction

<ipython-input-50-53c0f974067f>:9: RuntimeWarning: invalid value encountered in double_scalars
  return ratings_sum / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


,Title,Correlation,Genre
0,Prom Night,0.87,"[Horror, Mystery, Thriller]"
1,Wild Wild West,0.87,"[Action, Adventure, Comedy, Science Fiction, W..."
2,Blue Thunder,0.73,"[Science Fiction, Action, Thriller, Crime, Drama]"
3,Topaz,0.68,"[Action, Drama, Mystery, Thriller]"
4,Yamakasi - Les samouraïs des temps modernes,0.68,"[Action, Crime, Drama]"
5,Best Seller,0.67,"[Action, Crime, Drama, Thriller]"
6,Midnight in the Garden of Good and Evil,0.67,"[Crime, Drama, Mystery, Thriller]"
7,Big Bad Mama,0.64,"[Action, Comedy, Crime, Drama]"
8,The Enforcer,0.63,"[Action, Crime, Thriller]"
9,The River Wild,0.63,"[Action, Adventure, Crime, Thriller]"


In [54]:
# prediction using KNN
recommendations = recommend_knn_pearson('The Dark Knight', matrix, df, 10)
knn_prediction = pd.DataFrame(recommendations, columns = ['Title', 'Correlation', 'Genre'])
knn_prediction

<ipython-input-50-53c0f974067f>:9: RuntimeWarning: invalid value encountered in double_scalars
  return ratings_sum / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))


,Title,Correlation,Genre
0,Prom Night,0.873566,"[Horror, Mystery, Thriller]"
1,Wild Wild West,0.873566,"[Action, Adventure, Comedy, Science Fiction, W..."
2,Sneakers,0.615830,"[Comedy, Crime, Drama]"
3,Godzilla,0.613141,"[Science Fiction, Action, Thriller]"
4,Bushwhacked,0.596081,"[Adventure, Action, Comedy, Crime, Family]"
5,Ludwig,0.572718,"[Drama, History]"
6,El asaltante,0.546949,"[Drama, Foreign]"
7,Pirates of the Caribbean: On Stranger Tides,0.504551,"[Adventure, Action, Fantasy]"
8,Wet Hot American Summer,0.393004,[Comedy]
9,Man with the Screaming Brain,0.378676,"[Comedy, Horror, Science Fiction]"
